In [2]:
#| export

import jax
import jax.numpy as jnp
from jax import vmap, lax
from chewc.config import *

In [3]:
#| export


def _sample_chiasmata(
    key: jax.random.PRNGKey, map_length: jnp.ndarray, v_interference: float, max_crossovers: int
) -> jnp.ndarray:
    # Gamma gaps with shape=v_interference, scale=1/(2v), Haldane-like
    shape, scale = v_interference, 1.0 / (2.0 * v_interference)

    def scan_body(carry, _):
        key, last_pos = carry
        key, subkey = jax.random.split(key)
        new_pos = last_pos + jax.random.gamma(subkey, shape) * scale
        return (key, new_pos), new_pos

    key, initial_key = jax.random.split(key)
    init_carry = (key, jax.random.uniform(initial_key, minval=-1.0, maxval=0.0))
    _, crossover_positions = lax.scan(scan_body, init_carry, None, length=max_crossovers)
    return jnp.where((crossover_positions > 0) & (crossover_positions < map_length), crossover_positions, jnp.inf)


def _create_gamete_for_chromosome(
    key: jax.random.PRNGKey,
    parental_haplotypes: jnp.ndarray,   # (2, n_loci)
    locus_positions: jnp.ndarray,       # (n_loci,)
    v_interference: float,
    max_crossovers: int,
) -> jnp.ndarray:
    key, chiasma_key, hap_key = jax.random.split(key, 3)
    sorted_crossovers = jnp.sort(_sample_chiasmata(chiasma_key, locus_positions[-1], v_interference, max_crossovers))
    locus_segments = jnp.searchsorted(sorted_crossovers, locus_positions, side="right")
    start_hap = jax.random.choice(hap_key, jnp.array([0, 1], dtype=jnp.uint8))
    haplotype_choice = (start_hap + locus_segments) % 2
    return jnp.where(haplotype_choice == 0, parental_haplotypes[0], parental_haplotypes[1])


def _cross_pair(
    key: jax.random.PRNGKey,
    mother_geno: jnp.ndarray,  # (n_chr, 2, n_loci)
    father_geno: jnp.ndarray,  # (n_chr, 2, n_loci)
    locus_positions: jnp.ndarray,  # (n_chr, n_loci)
    n_chr: int,
    v_interference: float,
    max_crossovers: int,
) -> jnp.ndarray:
    key_mother, key_father = jax.random.split(key)
    vmapped_gamete_creator = vmap(_create_gamete_for_chromosome, in_axes=(0, 0, 0, None, None))
    mother_gamete = vmapped_gamete_creator(
        jax.random.split(key_mother, n_chr), mother_geno, locus_positions, v_interference, max_crossovers
    )
    father_gamete = vmapped_gamete_creator(
        jax.random.split(key_father, n_chr), father_geno, locus_positions, v_interference, max_crossovers
    )
    # stack as ploidy dimension=2
    return jnp.stack([mother_gamete, father_gamete], axis=1)  # (n_chr, 2, n_loci)


def create_offspring(
    key: jax.random.PRNGKey,
    parent_population: Population,
    pairings: jnp.ndarray,           # (n_crosses, 2) indices in parent_population
    s_config: StaticConfig,
    d_config: DynamicConfig,
    current_gen: int,
    next_id_start: int,
    max_crossovers: int = 20,
    v_interference: float = 2.5,
) -> Population:
    n_crosses, n_chr = pairings.shape[0], s_config.n_chr
    mother_indices, father_indices = pairings[:, 0], pairings[:, 1]

    mothers_geno = parent_population.geno[mother_indices]
    fathers_geno = parent_population.geno[father_indices]

    locus_positions = d_config.locus_positions

    offspring_geno = vmap(_cross_pair, in_axes=(0, 0, 0, None, None, None, None))(
        jax.random.split(key, n_crosses), mothers_geno, fathers_geno, locus_positions, n_chr, v_interference, max_crossovers
    )  # (n_crosses, n_chr, 2, n_loci)

    # IDs that continue from last generation
    offspring_ids = (next_id_start + jnp.arange(n_crosses, dtype=jnp.int32))[:, None]
    mother_ids = parent_population.meta[mother_indices, 0]
    father_ids = parent_population.meta[father_indices, 0]
    birth_gen = jnp.full((n_crosses, 1), current_gen, dtype=jnp.int32)

    return Population(
        geno=offspring_geno,
        meta=jnp.hstack([offspring_ids, jnp.stack([mother_ids, father_ids], axis=1), birth_gen]),
    )

In [4]:
#| hide
import nbdev; nbdev.nbdev_export()